In [10]:
import os
import pickle
import datetime
import pandas as pd
from src.db import DataBase
from src.utils_stocks import get_df_c
from src.utils_model import get_df_proba

DIR_DB = os.path.join(os.getcwd(), 'data', 'db')
DIR_MODELS = os.path.join(os.getcwd(), 'data', 'models')
db = DataBase([], DIR_DB)

f_model = 'tup_model_2021-02-05_2330.p'
with open(os.path.join(DIR_MODELS, f_model), 'rb') as f:
    tup_model = pickle.load(f)
    
ls_sym = ['TSLA']
date_str = '2021-02-04'
live_data = 0
target_profit, target_loss = 0.01, -0.03

In [11]:
def get_df_sym_filter(db):
    '''Returns dataframe of stock symbols in
    pre-selected sectors and industries
    Args:
        db (DataBase)
    '''
    ls_sec = [       
        'Technology',
        'Utilities',
        'Communication Services',
        #'Consumer Defensive',
        #'Consumer Cyclical',
        #'Energy',
        #'Basic Materials',
        #'Real Estate',
        #'Industrials,'
        #'Financial',
        #'Healthcare',
        #'Financial Services',
    ]
    ls_ind = [
        'Auto Manufacturers',
        'Internet Retail',
        'Education & Training Services',
        'Packaged Foods',
        'Grocery Stores',
    ]
    q = '''
        SELECT stocks.sym
               ,stocks.long_name
               ,stocks.sec
               ,stocks.ind
               ,stocks.quote_type
               ,stocks.fund_family
               ,stocks.summary
          FROM prices_d
          LEFT JOIN stocks
            ON stocks.sym = prices_d.sym
         WHERE DATE(date) = (SELECT MAX(DATE(date)) FROM prices_d WHERE sym='IBM')
           AND adj_close > 5
           AND volume > 100000
    '''
    df = pd.read_sql(q, db.conn)
    index = (df['sec'].isin(ls_sec))|(df['ind'].isin(ls_ind))
    df = df[index]
    df = df.reset_index()
    return df

In [16]:
df_sym = get_df_sym_filter(db)
ls_tup_buy, ls_tup_profit = [], []
for j, tup in df_sym.iloc[:100].iterrows():
    sym = tup['sym']
    print(j, sym)
    for i in range(2, 300, 2):
        try:
            df_c = get_df_c(sym, date_str, live_data, db, target_profit, target_loss, i)
            df_proba = get_df_proba(df_c, tup_model)
            df_proba = df_proba[df_proba['proba']>0.7]
            if not df_proba.empty:
                buy_index = df_proba.index[0] + 2
                df_c = get_df_c(sym, date_str, live_data, db, target_profit, target_loss)
                buy_price = df_c.iloc[buy_index]['close']
                df_c = df_c.iloc[buy_index:]
                df_c['profit'] = df_c['close']/buy_price-1
                ls_tup_buy.append(tup)
                if (df_c['profit']>0.01).any():
                    print(f'first buy index found: {sym} - {buy_index}, profit')
                    ls_tup_profit.append(tup)
                else:
                    print(f'first buy index found: {sym} - {buy_index}, not profit')
                break
        except Exception as e:
            print(f'Error: {sym} - {type(e).__name__} - {e}')
            break
len(ls_tup_profit)/len(ls_tup_buy)

0 SMAR
1 BILI
first buy index found: BILI - 58, profit
2 AVT
3 FISV
4 TUFN
Error: TUFN - Exception - Skip: Missing minute-level data for yesterday
5 KN
6 DISH
7 PLAN
8 CREE
9 FB
10 SHEN
Error: SHEN - Exception - 
11 CRTO
12 IMAX
13 PAYC
14 FE
Error: FE - Exception - Skip: NULL value in df_i columns (['close', 'sma9', 'sma180', 'vwap', 'sma9_var', 'sma180_var', 'vwap_var', 'prev_close_var', 'prev_floor_var', 'prev_ceil_var', 'prev1_candle_score', 'prev2_candle_score', 'prev3_candle_score'])
15 FVRR
16 UPLD
Error: UPLD - Exception - 
17 CSOD
18 UTZ
19 EVRG
20 ETSY
21 NKLA
22 OTTR
Error: OTTR - Exception - Skip: Missing minute-level data for yesterday
23 NTNX
24 DBD
25 EIGI
26 LQDT
first buy index found: LQDT - 90, profit
27 ABST
Error: ABST - Exception - Skip: Missing minute-level data for yesterday
28 CCC
Error: CCC - Exception - Skip: Missing minute-level data for yesterday
29 MODN
Error: MODN - Exception - Skip: Missing minute-level data for yesterday
30 CYBR
31 TDY
32 IMXI
Error: IMX

In [17]:
len(ls_tup_profit)/len(ls_tup_buy)

0.8333333333333334